In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests # For sending GET requests from the API
import json # For dealing with json responses we receive from the API
import csv 
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
import time #To add wait time between requests

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis-dataset/twitter_dataset_caa.csv


# **To-Do Lost**
* Upate date-time before running
* Start to End time difference of 6 days works

# **Note**

* Links mentioned in tweets should be analysed separately as the link may convey the user's intent
* Didn't collect tweets over extended period of time because of limited Twitter Dev Perms and,
* Labelling of such a huge dataset becomes super complex (even more so than it already was)
* Have to manually change type of "id" column of the dataset in the csv file because it keeps defaulting to scientific number instead of a string, so all id's get rounded off to essentially the same tweet id
* Many tweets seem like duplicates, but they're not, they have just been retweeted. This is important to the dataset and haven't been removed because they're indicative of how the public feels to other people's opinions as well

In [2]:
# Create file
csvFile = open("data.csv", "w", newline="", encoding='utf-8')

csvFile.close()

Wipes the contents of data.csv on every run because of write mode

In [3]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAABk4igEAAAAAuWtFacMdxS%2F7CSfS9fM0qKBECAw%3DE5fUtU64PorcKd7WCprHUlisk3WtUDugb4t4FKSnJDeftHABMb'

Dictionary containing key-value = user's_environment_variable-corresponding_value

In [4]:
def auth():
    return os.getenv('TOKEN')

Environment variable value of TOKEN is returned

In [5]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

Function to take the bearer token, pass it for authorization and return headers to access the API

In [6]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

* Request for the endpoint (limited because of my Twitter Dev Perms) and the parameters
* The ‘next_token’ parameter is used to get the next ‘page’ of results

In [7]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

* Connecting to the endpoint
* The function sends the “GET” request and if everything is correct (response code 200), it returns the response in “JSON” format

next_token is set to “None” by default since I only care about it if it exists

In [8]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "caa india lang:en"
start_time = "2022-11-06T00:00:00.000Z"
end_time = "2022-11-12T00:00:00.000Z"
max_results = 100

* Keywords used - "caa" and "india"
* Date-time needs to be updated on each run because of my limited Twitter Dev Perms

In [9]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [10]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "113955787",
            "conversation_id": "1586107220399427585",
            "created_at": "2022-10-28T21:26:44.000Z",
            "edit_history_tweet_ids": [
                "1586107220399427585"
            ],
            "id": "1586107220399427585",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 13
            },
            "referenced_tweets": [
                {
                    "id": "1586015935499943937",
                    "type": "retweeted"
                }
            ],
            "reply_settings": "everyone",
            "source": "Twitter for Android",
            "text": "RT @Informaticafan: @ReallySwara @RahulGandhi @bharatjodo Credit where it's due.Inspite of not reading  draft of NRC not reading about CAA\u2026"
        },
        {
            "author_id": "71813022

The response is basically read as a Python dictionary and the keys are either "data" or "meta"
* "meta" obviously contains metadata about the obtained tweets

In [11]:
json_response['data'][0]['created_at']

'2022-10-28T21:26:44.000Z'

In [12]:
json_response['meta']['result_count']

100

In [13]:
with open('data.json', 'w') as f:
    json.dump(json_response, f)

File saved in JSON format as is

In [14]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvFile.close()

File reopened in "append" mode so that presaved data isn't overwritten

In [15]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that
 
        # 1. Tweet ID
        tweet_id = tweet['id']

        # 2. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [tweet_id, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

Inputs the response and desired filename into this function, and the function will append all the data collected to the CSV file

In [16]:
append_to_csv(json_response, "data.csv")

# of Tweets added from this response:  100


In [17]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "caa india lang:en"
start_list =    ['2022-10-25T00:00:00.000Z',
                 '2022-10-25T12:00:00.000Z',
                 '2022-10-26T00:00:00.000Z',
                 '2022-10-26T12:00:00.000Z',
                 '2022-10-27T00:00:00.000Z',
                 '2022-10-27T12:00:00.000Z',
                 '2022-10-28T00:00:00.000Z',
                 '2022-10-28T12:00:00.000Z']

end_list =      ['2022-10-25T12:00:00.000Z',
                 '2022-10-26T00:00:00.000Z',
                 '2022-10-26T12:00:00.000Z',
                 '2022-10-27T00:00:00.000Z',
                 '2022-10-27T12:00:00.000Z',
                 '2022-10-28T00:00:00.000Z',
                 '2022-10-28T12:00:00.000Z',
                 '2022-10-29T00:00:00.000Z']
max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzel4rxpflbwer0xlxl782x20k6m5
Start Date:  2022-10-25T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  100
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzel4w5d488fjvia7tlbp9b8syhdp
Start Date:  2022-10-25T12:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  200
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-26T00:00:00.000Z
# of Tweets added from this response:  84
Total # of Tweets added:  284
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-26T12:00:00.000Z
# of Tweets added from this response:  86
Total # of Tweets added:  370
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
---------------

* For-loop that goes over the months/weeks/days we want to cover (Depending on how it is set) for better distributed data
* While-loop that controls the maximum number of tweets to be collected per time period
* time.sleep() added between calls to ensure API isn't spammed with requests

In [18]:
df_state = pd.read_csv("./data.csv")
df_state.shape

(558, 2)

In [19]:
# Number of duplicates
df_state.duplicated().sum()

99

In [20]:
# Removing duplicate rows in the dataframe
DF_RM_DUP = df_state.drop_duplicates(keep='first')
DF_RM_DUP.shape

(459, 2)

In [21]:
with open('twitter_dataset_caa.csv', 'w') as csv_file:
    DF_RM_DUP.to_csv(path_or_buf=csv_file, index = False)

In [22]:
from sklearn.model_selection import train_test_split
# Splitting training and test data

tweets = pd.read_csv("../input/twitter-sentiment-analysis-dataset/twitter_dataset_caa.csv")
train_set, test_set = train_test_split(tweets, test_size = 0.2, random_state = 1)
print(f"Rows in training set: {len(train_set)}\nRows in testing set: {len(test_set)}\n")

Rows in training set: 502
Rows in testing set: 126



In [27]:
train_set.head()

,1.59E+18,RT @Informaticafan: @ReallySwara @RahulGandhi @bharatjodo Credit where it's due.Inspite of not reading draft of NRC not reading about CAA…
314,1.580000e+18,RT @MehboobaMufti: Proud moment that UK will h...
500,1.590000e+18,@HappymonJacob Slight correction :\nIndian-ori...
350,1.580000e+18,What has @ShekharGupta said about the #CAA #NR...
80,1.590000e+18,@Wine_Sommelier_ @bakshi_arnab @onty4u I dont ...
241,1.580000e+18,RT @MehboobaMufti: Proud moment that UK will h...


In [28]:
train_set.to_csv("train.csv", index = False)

In [29]:
test_set.head()

,1.59E+18,RT @Informaticafan: @ReallySwara @RahulGandhi @bharatjodo Credit where it's due.Inspite of not reading draft of NRC not reading about CAA…
496,1.590000e+18,@ANI @ArvindKejriwal \nFirst support Hindu pop...
107,1.580000e+18,RT @the_hindu: The #CAA seeks to grant #citize...
66,1.590000e+18,@Elizatweetz There are many like her. How does...
159,1.580000e+18,RT @aayeff1: @KewalChopra6 @sunild1909 @TanyaG...
511,1.590000e+18,"@MehboobaMufti Giving eg, of @RishiSunak u poi..."


In [30]:
test_set.to_csv("test.csv", index = False)